Organize you code as per the below Unified Modelling Language (UML) diagram in Figure 2. Furthermore, consider the points listed below and discuss these points in a section labelled Part 4 in your report (where appropriate). 

* Instead of re-writing A* algorithm for this part, treat the class from UML as an “adapter”. 
* Discuss what design principles and patterns are being used in the diagram
* The UML is limited in the sense that graph nodes are represented by the integers. How would you alter the UML diagram to accommodate various needs such as nodes being represented Strings or carrying more information than their names.? Explain how you would change the design in Figure 2 to be robust to these potential changes.
* Discuss what other types of graphs we could have implement “Graph”. What other implementations exist?

In [12]:
from abc import ABC, abstractmethod
from typing import Dict, Tuple, List

In [16]:
class Graph(ABC):
    def __init__(self):
        self.adj = {}
        self.weights = {}
    def get_adj_nodes(self, node):
        return self.adj[node]
    def add_node(self, node):
        self.adj[node] = []
    def add_edge(self, node1, node2, weight):
        if node2 not in self.adj[node1]:
            self.adj[node1].append(node2)
        self.weights[(node1, node2)] = weight
    def get_num_of_nodes(self):
        return len(self.adj)
    @abstractmethod
    def w(self, node1, node2):
        pass

In [26]:
class WeightedGraph(Graph):
    def __init__(self):
        super().__init__()
        self.adj = {}
        self.weights = {}
    def w(self, node1, node2):
        if self.are_connected(node1, node2):
            return self.weights[(node1, node2)]
    def add_edge(self, start: int, end: int, w: float):
        super().add_edge(start, end, w)

In [62]:
class HeuristicGraph(WeightedGraph):
    def __init__(self, heuristic):
        super().__init__()
        self.heuristic_dict = heuristic  # Store heuristic values in a dictionary

    def heuristic(self, node, destination):  # Corrected method to access heuristic values
        return self.heuristic_dict[node]

    def get_heuristic(self) -> Dict[int, float]:
        return self.heuristic_dict


In [63]:
class ShortPathFinder:

    def __init__(self):
        self.Graph = Graph()
        self.SPAlgorithm = SPAlgorithm()

    def calc_short_path(self, source, dest):
        dist = self.SPAlgorithm(self.Graph, source, dest)
        total = 0
        for key in dist.keys():
            total += dist[key]
        return total

    @property
    def set_graph(self):
        self._Graph

    @set_graph.setter
    def set_graph(self, graph):
        self._Graph = graph

    @property
    def set_algorithm(self,):
        self._SPAlgorithm
    
    @set_algorithm.setter
    def set_algorithm(self, algorithm):
        self._SPAlgorithm = algorithm

In [64]:
class SPAlgorithm(ABC):
    def __init__(g):
        self.g = g
    @abstractmethod
    def calc_sp(self, source: int, dst: int):
        pass

In [65]:
# min pq necessary for djisktra's 
class Item:
    def __init__(self, value, key):
        self.value = value
        self.key = key

class MinHeap:
    def __init__(self, elements):
        self.heap = elements
        self.positions = {element.value: i for i, element in enumerate(elements)}
        self.size = len(elements)
        self.build_heap()

    def parent(self, i):
        return (i - 1) // 2

    def left(self, i):
        return 2 * i + 1

    def right(self, i):
        return 2 * i + 2

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        self.positions[self.heap[i].value], self.positions[self.heap[j].value] = i, j

    def min_heapify(self, i):
        l = self.left(i)
        r = self.right(i)
        smallest = i
        if l < self.size and self.heap[l].key < self.heap[i].key:
            smallest = l
        if r < self.size and self.heap[r].key < self.heap[smallest].key:
            smallest = r
        if smallest != i:
            self.swap(i, smallest)
            self.min_heapify(smallest)

    def build_heap(self):
        for i in range(self.size // 2, -1, -1):
            self.min_heapify(i)

    def extract_min(self):
        min_element = self.heap[0]
        self.size -= 1
        self.heap[0] = self.heap[self.size]
        self.positions[self.heap[0].value] = 0
        self.heap.pop()
        self.min_heapify(0)
        return min_element

    def decrease_key(self, value, new_key):
        i = self.positions[value]
        if new_key < self.heap[i].key:
            self.heap[i].key = new_key
            while i > 0 and self.heap[self.parent(i)].key > self.heap[i].key:
                self.swap(i, self.parent(i))
                i = self.parent(i)

    def insert(self, element):
        self.size += 1
        self.heap.append(element)
        self.positions[element.value] = self.size - 1
        self.decrease_key(element.value, element.key)

    def is_empty(self):
        return self.size == 0

In [66]:
class Dijkstra(SPAlgorithm):
    def __init__(self, g):
        self.g = g

    def calc_sp(self, graph, source, destination):
        visited = {node: False for node in range(graph.get_number_of_nodes())}
        distance = {node: float('inf') for node in range(graph.get_number_of_nodes())}

        Q = MinHeap([])

        for i in range(graph.get_number_of_nodes()):
            Q.insert(Item(i, float('inf')))

        Q.decrease_key(source, 0)
        distance[source] = 0

        while not Q.is_empty():
            current_node = Q.extract_min().value
            visited[current_node] = True

            if current_node == destination:
                break

            for neighbor in graph.graph[current_node]:
                edge_weight = graph.get_weights(current_node, neighbor)
                temp = distance[current_node] + edge_weight

                if not visited[neighbor] and temp < distance[neighbor]:
                    distance[neighbor] = temp
                    Q.decrease_key(neighbor, temp)

        return distance[destination] if distance[destination] != float('inf') else None

In [67]:
class Bellman_Ford(SPAlgorithm):
    def __init__(self, g):
        self.g = g

    def calc_sp(self, source, dst):
        dist = {node: float('inf') for node in self.g.adj}
        dist[source] = 0
        paths = {}
        for i in range(len(self.g.adj) - 1):
            for u in self.g.adj:
                for v in self.g.adj[u]:
                    if dist[u] + self.g.w(u, v) < dist[v]:
                        dist[v] = dist[u] + self.g.w(u, v)
                        paths[v] = paths.get(u, []) + [v]

        for u in self.g.adj:
            for v in self.g.adj[u]:
                if dist[u] + self.g.w(u, v) < dist[v]:
                    raise ValueError("Graph contains negative weight cycle")

        return dist, paths

In [71]:
class AStarAdapter(SPAlgorithm):
    def __init__(self, graph):
        self.graph = graph

    def calc_sp(graph, source, destination):
        h = {node: float(self.graph.heuristic(node, destination)) for node in self.graph.adj.keys()}
        openset = MinHeap([])
        openset.insert(Item(source, float('inf')))
        came_from = {}
        g_score = {node: float('inf') for node in self.graph.adj.keys()}
        g_score[source] = 0
        f_score = {node: float('inf') for node in self.graph.adj.keys()}
        f_score[source] = h[source]

        while not openset.is_empty():
            current = openset.extract_min().value
            if current == destination:
                break
            for neighbour in self.graph.adj[current]:
                t_gscore = g_score[current] + self.graph.weights[(current, neighbour)]
                if t_gscore < g_score[neighbour]:
                    came_from[neighbour] = current
                    g_score[neighbour] = t_gscore
                    f_score[neighbour] = t_gscore + h[neighbour]
                    if neighbour not in came_from:
                        openset.insert(Item(neighbour, f_score[neighbour]))

        shortest_path = []
        current = destination
        while current in came_from:
            shortest_path.insert(0, current)
            current = came_from[current]
        shortest_path.insert(0, source)

        return came_from, shortest_path


def test_a_star_adapter():
    # Test case 1: Simple path
    graph1 = HeuristicGraph({0: 2, 1: 1, 2: 0})
    graph1.add_node(0)  # Add nodes to the graph first
    graph1.add_node(1)
    graph1.add_node(2)
    graph1.add_edge(0, 1, 1)  # Then add edges
    graph1.add_edge(1, 2, 1)
    a_star1 = AStarAdapter(graph1)
    print("Test case 1:")
    print(a_star1.calc_sp(0, 2))

    # Test case 2: Complex path
    graph2 = HeuristicGraph({0: 3, 1: 2, 2: 1, 3: 0})
    graph2.add_node(0)
    graph2.add_node(1)
    graph2.add_node(2)
    graph2.add_node(3)
    graph2.add_edge(0, 1, 1)
    graph2.add_edge(1, 2, 1)
    graph2.add_edge(2, 3, 1)
    a_star2 = AStarAdapter(graph2)
    print("\nTest case 2:")
    print(a_star2.calc_sp(0, 3))

    # Test case 3: No path
    graph3 = HeuristicGraph({0: 2, 1: 3, 2: 4, 3: 5})
    graph3.add_node(0)
    graph3.add_node(1)
    graph3.add_node(2)
    graph3.add_node(3)
    a_star3 = AStarAdapter(graph3)
    print("\nTest case 3:")
    print(a_star3.calc_sp(0, 3))

    # Test case 4: Graph with loops
    graph4 = HeuristicGraph({0: 2, 1: 3, 2: 4, 3: 5})
    graph4.add_node(0)
    graph4.add_node(1)
    graph4.add_node(2)
    graph4.add_node(3)
    graph4.add_edge(0, 0, 1)  # Loop
    graph4.add_edge(2, 2, 1)  # Loop
    a_star4 = AStarAdapter(graph4)
    print("\nTest case 4:")
    print(a_star4.calc_sp(0, 3))

    print("\nAll test cases executed.")

# Run the test cases
test_a_star_adapter()



Test case 1:
({1: 0}, [0])

Test case 2:
({1: 0}, [0])

Test case 3:
({}, [0])

Test case 4:
({}, [0])

All test cases executed.
